###  It is required to import UMLS model into Neo4J before proceeding with the code

You can refer to the Github code to import UMLS data into Neo4J :  

https://github.com/MIS-GrApH-AI/Public/blob/main/Data_import/UMLS_2020AB.ipynb

# Importing MED-RT into UMLS model

To download the MED-RT data visit the link -  https://evs.nci.nih.gov/ftp1/MED-RT/

For our code we are using the XML.zip files. 

MED-RT documentation can be read from here - https://evs.nci.nih.gov/ftp1/MED-RT/MED-RT%20Documentation.pdf

###  Refer to the procedure to get Association.csv

Use the following codes in your Neo4J browser : (You must change the file retrieve location according to where you have stored the MED-RT XML file)




Code 1:

WITH "CALL apoc.load.xml(\"file:///C:\\Users\\pc\\Desktop\\C-BIA\\Healthcare\\MED-RT\\Database\\Core_MEDRT_XML\\Parts\\Core_MEDRT_2021.07.06_XML.xml\" , '/terminology/association') YIELD value as association WITH [attr IN association._children WHERE attr._type IN ['namespace','name','from_namespace','from_name','from_code','to_namespace','to_name','to_code'] | attr._text] as pairs, [item in association._children WHERE item._type = \"qualifier\"] AS q1 RETURN pairs[0] as namespace, pairs[1] as name, pairs[2] as from_namespace, pairs[3] as from_name, pairs[4] as from_code, pairs[5] as to_namespace, pairs[6] as to_name, pairs[7] as to_code, q1[0]._children[0]._text as qualifier0_namespace, q1[0]._children[1]._text as qualifier0_name, q1[0]._children[2]._text as qualifier0_value, q1[1]._children[0]._text as qualifier1_namespace, q1[1]._children[1]._text as qualifier1_name, q1[1]._children[2]._text as qualifier1_value; " as query
CALL apoc.export.csv.query(query, "association.csv", {})
YIELD file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data
RETURN file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data;

Once this action is performed, "association.csv" will be created in your Neo4J dbms folder.

It is recommended to move this file to another easily accessible location since we will be referencing it in the upcoming code 

Similar actions must be performed for Code 2 given below

Code 2:

WITH "CALL apoc.load.xml(\"file:///C:\\Users\\pc\\Desktop\\C-BIA\\Healthcare\\MED-RT\\Database\\Core_MEDRT_XML\\Parts\\Core_MEDRT_2021.07.06_XML.xml\" , '/terminology/namespace') YIELD value as namespace WITH  [attr IN namespace._children WHERE attr._type IN ['name','code','version','authority'] | attr._text] as pairs RETURN  pairs[0] as name, pairs[1] as code, pairs[2] as version, pairs[3] as authority;" as query
CALL apoc.export.csv.query(query, "namespace.csv", {})
YIELD file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data
RETURN file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data;


The name of the csv file produced is "namespace.csv"

In [66]:
import pandas as pd

In [67]:
import numpy as np

In [68]:
# Load MRCONSO.RRF into a dataframe
mrconso = pd.read_csv('C:/Users/pc/Desktop/C-BIA/Healthcare/UMLS/UMLS_Data/2021AA/META/MRCONSO.RRF', sep='|', header=None, encoding='utf-8')
mrconso[:5]

C:\Users\pc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,C0000005,ENG,P,L0000005,PF,S0007492,Y,A26634265,NaN,M0019694,D012711,MSH,PEP,D012711,(131)I-Macroaggregated Albumin,0,N,256.0,NaN
1,C0000005,ENG,S,L0270109,PF,S0007491,Y,A26634266,NaN,M0019694,D012711,MSH,ET,D012711,(131)I-MAA,0,N,256.0,NaN
2,C0000039,ENG,P,L0000039,PF,S17175117,N,A28315139,9194921.0,1926948,NaN,RXNORM,IN,1926948,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN
3,C0000039,ENG,P,L0000039,PF,S17175117,Y,A28572604,NaN,NaN,NaN,MTH,PN,NOCODE,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN
4,C0000039,ENG,P,L0000039,VC,S0007564,Y,A0016515,NaN,M0023172,D015060,MSH,MH,D015060,"1,2-Dipalmitoylphosphatidylcholine",0,N,NaN,NaN


In [69]:
mrconso.columns = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS', 'CVF', '']

In [70]:
mrconso.drop(labels=['SUPPRESS', 'CODE', 'CVF','SAUI','SDUI', ''], axis=1, inplace=True)

In [71]:
mrconso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12139019 entries, 0 to 12139018
Data columns (total 13 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   CUI     object
 1   LAT     object
 2   TS      object
 3   LUI     object
 4   STT     object
 5   SUI     object
 6   ISPREF  object
 7   AUI     object
 8   SCUI    object
 9   SAB     object
 10  TTY     object
 11  STR     object
 12  SRL     int64 
dtypes: int64(1), object(12)
memory usage: 1.2+ GB


In [72]:
mrconso[:5]

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SCUI,SAB,TTY,STR,SRL
0,C0000005,ENG,P,L0000005,PF,S0007492,Y,A26634265,M0019694,MSH,PEP,(131)I-Macroaggregated Albumin,0
1,C0000005,ENG,S,L0270109,PF,S0007491,Y,A26634266,M0019694,MSH,ET,(131)I-MAA,0
2,C0000039,ENG,P,L0000039,PF,S17175117,N,A28315139,1926948,RXNORM,IN,"1,2-dipalmitoylphosphatidylcholine",0
3,C0000039,ENG,P,L0000039,PF,S17175117,Y,A28572604,NaN,MTH,PN,"1,2-dipalmitoylphosphatidylcholine",0
4,C0000039,ENG,P,L0000039,VC,S0007564,Y,A0016515,M0023172,MSH,MH,"1,2-Dipalmitoylphosphatidylcholine",0


### Testing Merge - MRCONSO on Association


#### We perform merging of the association file to  

The files "association.csv" and "namespace.csv" file locations must be referenced here 

In [73]:
#Association file is used for merging MRCONSO (UMLS) with MED-RT data
association = pd.read_csv(r'C:\Users\pc\Desktop\C-BIA\Healthcare\MED-RT\XML - CSV\association.csv')

In [74]:
#Version file is used for referencing the current MED-RT version into our UMLS model
version = pd.read_csv(r'C:\Users\pc\Desktop\C-BIA\Healthcare\MED-RT\XML - CSV\namespace.csv')

In [75]:
association['from_namespace_rectified'] = association['from_namespace']
association['to_namespace_rectified'] = association['to_namespace']

In [76]:
association['from_namespace_rectified'] = association['from_namespace_rectified'].replace('RxNorm','RXNORM')
association['from_namespace_rectified'] = association['from_namespace_rectified'].replace('MeSH','MSH')
association['from_namespace_rectified'] = association['from_namespace_rectified'].replace('SNOMED CT','SNOMEDCT_US')

In [77]:
association['to_namespace_rectified'] = association['to_namespace_rectified'].replace('RxNorm','RXNORM')
association['to_namespace_rectified'] = association['to_namespace_rectified'].replace('MeSH','MSH')
association['to_namespace_rectified'] = association['to_namespace_rectified'].replace('SNOMED CT','SNOMEDCT_US')

In [78]:
mrconsosub = mrconso[['SCUI','SAB','CUI']]

In [79]:
mrconsosub.drop_duplicates(subset=None, keep="first", inplace=True)

<ipython-input-79-fd34af682f5c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrconsosub.drop_duplicates(subset=None, keep="first", inplace=True)


In [80]:
mrconsosub.to_csv('mrconsosub.csv', index=False, header=1, encoding='utf-8')

In [81]:
asso_merge = pd.merge(association, mrconsosub, left_on=['from_code','from_namespace_rectified'] , right_on=['SCUI','SAB'],  how='left')

In [82]:
asso_merge['from_CUI'] = asso_merge['CUI']

In [83]:
asso_merge['from_SAB'] = asso_merge['SAB']

In [84]:
asso_merge['from_SCUI'] = asso_merge['SCUI']

In [85]:
asso_merge.drop(labels=['SCUI','SAB','CUI'], axis=1, inplace=True)

In [86]:
asso_merge.drop_duplicates()

,namespace,name,from_namespace,from_name,from_code,to_namespace,to_name,to_code,qualifier0_namespace,qualifier0_name,qualifier0_value,qualifier1_namespace,qualifier1_name,qualifier1_value,from_namespace_rectified,to_namespace_rectified,from_CUI,from_SAB,from_SCUI
0,MED-RT,Parent Of,MED-RT,Cellular or Molecular Interactions [MoA],N0000000223,MED-RT,Unknown Cellular or Molecular Interaction [MoA],N0000009915,MED-RT,Authority,MEDRT,NaN,NaN,NaN,MED-RT,MED-RT,C1373094,MED-RT,N0000000223
1,MED-RT,Parent Of,MED-RT,Cellular or Molecular Interactions [MoA],N0000000223,MED-RT,Immunologic and Biological Factor Interactions...,N0000000254,MED-RT,Authority,MEDRT,NaN,NaN,NaN,MED-RT,MED-RT,C1373094,MED-RT,N0000000223
2,MED-RT,Parent Of,MED-RT,Cellular or Molecular Interactions [MoA],N0000000223,MED-RT,Gene Expression Interactions [MoA],N0000193793,MED-RT,Authority,MEDRT,NaN,NaN,NaN,MED-RT,MED-RT,C1373094,MED-RT,N0000000223
3,MED-RT,Parent Of,MED-RT,Cellular or Molecular Interactions [MoA],N0000000223,MED-RT,Biological Macromolecular Activity [MoA],N0000000142,MED-RT,Authority,MEDRT,NaN,NaN,NaN,MED-RT,MED-RT,C1373094,MED-RT,N0000000223
4,MED-RT,Parent Of,MED-RT,Cellular or Molecular Interactions [MoA],N0000000223,MED-RT,Receptor Interactions [MoA],N0000000085,MED-RT,Authority,MEDRT,NaN,NaN,NaN,MED-RT,MED-RT,C1373094,MED-RT,N0000000223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88987,MED-RT,Synonym Of,RxNorm,fluocortolone pivalate,4465,MeSH,Fluocortolone Pivalate,M0008596,MED-RT,Authority,NLMUMLS,NaN,NaN,NaN,RXNORM,MSH,C0016304,RXNORM,4465
88988,MED-RT,Synonym Of,RxNorm,azelaic acid,18602,MeSH,azelaic acid,M0055275,MED-RT,Authority,NLMUMLS,NaN,NaN,NaN,RXNORM,MSH,C0052761,RXNORM,18602
88989,MED-RT,Synonym Of,RxNorm,tazarotene,83947,MeSH,tazarotene,M0230113,MED-RT,Authority,NLMUMLS,NaN,NaN,NaN,RXNORM,MSH,C0288792,RXNORM,83947
88990,MED-RT,Synonym Of,RxNorm,ammonium lactate,215247,MeSH,Ammonium Lactate,M0582740,MED-RT,Authority,NLMUMLS,NaN,NaN,NaN,RXNORM,MSH,C0718495,RXNORM,215247


In [87]:
asso_merge_final = pd.merge(asso_merge, mrconsosub, left_on=['to_code','to_namespace_rectified'] , right_on=['SCUI','SAB'],  how='left')

In [88]:
asso_merge_final['to_CUI'] = asso_merge_final['CUI'] 

In [89]:
asso_merge_final['to_SAB'] = asso_merge_final['SAB']

In [90]:
asso_merge_final['to_SCUI'] = asso_merge_final['SCUI']

In [91]:
asso_merge_final.drop(labels=['SCUI','SAB','CUI','from_SAB','to_SAB','from_SCUI','to_SCUI'], axis=1, inplace=True)

In [92]:
vers = version['version'].astype(str)
asso_merge_final['version'] = vers[0]
#asso_merge_final['version'] = asso_merge_final['version'].replace(asso_merge_final['version'],vers[0])

In [93]:
asso_merge_final.to_csv('Association_Merged_MEDRT.csv', index=False, header=1, encoding='utf-8')

##### Move the "Association_Merged_MEDRT" into the Neo4J import folder

### Creating relationships of MED-RT data into UMLS model

In [94]:
import pandas as pd

In [95]:
import getpass
password = getpass.getpass("\nPlease enter the Neo4j database password to continue \n")

In [96]:
from neo4j import GraphDatabase
driver=GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j',password))
session=driver.session()

In [97]:
# Create a node for each concept in the UMLS
#query = '''USING PERIODIC COMMIT 100000 LOAD CSV FROM "file:///CUIs_preferred_terms.csv" AS COLUMN CREATE (:Concept_UMLS {preferred_term:COLUMN[1], cui:COLUMN[0], semantic_type:COLUMN[2], UMLS_edition:'2021AA'})'''

#session.run(query)

In [98]:
# Create a uniqueness constraint for the CUI property of each Concept_UMLS node
#query = 'CREATE CONSTRAINT UniqueCUIforConceptConstraint ON (c:Concept_UMLS) ASSERT c.cui IS UNIQUE'
#session.run(query)

In [99]:
# Create the UMLS string nodes
#query = '''USING PERIODIC COMMIT 100000 LOAD CSV FROM "file:///str_to_CUI.csv" AS COLUMN CREATE (s:String_UMLS {string:COLUMN[0], cui:COLUMN[1], semantic_type:COLUMN[2]})'''

#session.run(query)

In [16]:
command = '''USING PERIODIC COMMIT 100000 LOAD CSV WITH HEADERS FROM "file:///MeSH_Association_Merged.csv" AS COLUMN MATCH (c1:Concept_UMLS {cui:COLUMN.from_CUI}) MATCH (c2:Concept_UMLS {cui:COLUMN.to_CUI}) OPTIONAL MATCH (c1)-[rel]->(c2) WITH c1, c2, COLUMN, COLLECT(TYPE(rel)) AS relTypes WHERE NOT COLUMN.name IN relTypes CALL apoc.create.relationship(c1, COLUMN.name, {source:COLUMN.namespace, version:COLUMN.version}, c2) YIELD rel as newrel RETURN c1, c2, newrel;'''
session.run(command)

### If you want to delete self loop relation - Synonym_Of 

###### Note - This can be used to remove any other self loop relationships as well, just replace 'Synonym Of' with the relationship name 

In [101]:
#Delete self-loop relationship with name "Synonym_Of"

#command = '''MATCH p=(n1)-[r:`Synonym Of`]->(n2) WHERE n1.cui = n2.cui DELETE r'''
#session.run(command)

###### Once you reach this point , you can delete all the existing variables. It is preferable to do so if your system cannot handle executing all of these codes in one go

# Importing MeSH into UMLS model

To download the MeSH data visit the link -  https://nlmpubs.nlm.nih.gov/projects/mesh/MESH_FILES/xmlmesh/

For our code we are using desc2021.xml and supp2021.xml 

More information on MeSH can be found here - https://www.nlm.nih.gov/mesh/meshhome.html

### Refer to the procedure to get MeSH_Association_desc.csv  

Use the following codes in your Neo4J browser : (You must change the file retrieve location according to where you have stored the MeSH XML file)

Code 1:

WITH "CALL apoc.load.xml(\"file:///C:\\Users\\pc\\Desktop\\C-BIA\\Healthcare\\MeSH\\XML files\\Data\\desc2021.xml\" , '/DescriptorRecordSet/DescriptorRecord') YIELD value UNWIND value as desrec WITH [item in desrec._children WHERE item._type = \"DescriptorUI\"] AS Q, [item in desrec._children WHERE item._type = \"ConceptList\"] AS conceptlist1, desrec AS desrec UNWIND conceptlist1 AS conceptlist WITH [item in conceptlist._children WHERE item._type = \"Concept\"] AS concept1, Q AS Q, desrec AS desrec, conceptlist AS conceptlist UNWIND concept1 AS concept WITH [item in concept._children WHERE item._type = \"ConceptUI\"] AS ConceptUI, [item in concept._children WHERE item._type = \"ConceptRelationList\"] AS CRL1, Q AS Q, desrec AS desrec, conceptlist AS conceptlist, concept AS concept UNWIND CRL1 AS CRL WITH [item in CRL._children WHERE item._type = \"ConceptRelation\"] AS CR1, Q AS Q, desrec AS desrec, conceptlist AS conceptlist, concept AS concept,  ConceptUI AS  ConceptUI, CRL AS CRL UNWIND CR1 AS CR WITH [item in CR._children WHERE item._type = \"Concept1UI\"] AS C1UI, [item in CR._children WHERE item._type = \"Concept2UI\"] AS C2UI, Q AS Q, desrec AS desrec, conceptlist AS conceptlist, concept AS concept,  ConceptUI AS  ConceptUI, CRL AS CRL, CR AS CR RETURN DISTINCT \"MeSH\" AS namespace, CR.RelationName AS name, \"MeSH\" AS from_namespace, C1UI[0]._text AS from_code, \"MeSH\" AS to_namespace, C2UI[0]._text AS to_code, \"2021_2021.01.01\" AS version;" as query
CALL apoc.export.csv.query(query, "MeSH_association_desc.csv", {})
YIELD file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data
RETURN file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data;


Once this action is performed, "MeSH_association_desc.csv" will be created in your Neo4J dbms folder.

It is recommended to move this file to another easily accessible location since we will be referencing it in the upcoming code 

Code 2:

WITH "CALL apoc.load.xml(\"file:///C:\\Users\\pc\\Desktop\\C-BIA\\Healthcare\\MeSH\\XML files\\Data\\supp2021.xml\" , '/SupplementalRecordSet/ SupplementalRecord') YIELD value UNWIND value as desrec WITH [item in desrec._children WHERE item._type = \"SupplementalRecordUI\"] AS Q, [item in desrec._children WHERE item._type = \"ConceptList\"] AS conceptlist1, desrec AS desrec UNWIND conceptlist1 AS conceptlist WITH [item in conceptlist._children WHERE item._type = \"Concept\"] AS concept1, Q AS Q, desrec AS desrec, conceptlist AS conceptlist UNWIND concept1 AS concept WITH [item in concept._children WHERE item._type = \"ConceptUI\"] AS ConceptUI, [item in concept._children WHERE item._type = \"ConceptRelationList\"] AS CRL1, Q AS Q, desrec AS desrec, conceptlist AS conceptlist, concept AS concept UNWIND CRL1 AS CRL WITH [item in CRL._children WHERE item._type = \"ConceptRelation\"] AS CR1, Q AS Q, desrec AS desrec, conceptlist AS conceptlist, concept AS concept,  ConceptUI AS  ConceptUI, CRL AS CRL UNWIND CR1 AS CR WITH [item in CR._children WHERE item._type = \"Concept1UI\"] AS C1UI, [item in CR._children WHERE item._type = \"Concept2UI\"] AS C2UI, Q AS Q, desrec AS desrec, conceptlist AS conceptlist, concept AS concept,  ConceptUI AS  ConceptUI, CRL AS CRL, CR AS CR RETURN DISTINCT \"MeSH\" AS namespace, CR.RelationName AS name, \"MeSH\" AS from_namespace, C1UI[0]._text AS from_code, \"MeSH\" AS to_namespace, C2UI[0]._text AS to_code, \"2021_2021.01.01\" AS version;" as query
CALL apoc.export.csv.query(query, "MeSH_association_supp.csv", {})
YIELD file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data
RETURN file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data;


###### Code 2 requires a higher amount of heap space. It is suggested to perform the steps below used on MeSH_association_desc for MeSH_association_supp as well

In [102]:
import pandas as pd

In [103]:
import numpy as np

In [104]:
# Load MRCONSO.RRF into a dataframe
mrconso = pd.read_csv('C:/Users/pc/Desktop/C-BIA/Healthcare/UMLS/UMLS_Data/2021AA/META/MRCONSO.RRF', sep='|', header=None, encoding='utf-8')
mrconso[:5]

C:\Users\pc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,C0000005,ENG,P,L0000005,PF,S0007492,Y,A26634265,NaN,M0019694,D012711,MSH,PEP,D012711,(131)I-Macroaggregated Albumin,0,N,256.0,NaN
1,C0000005,ENG,S,L0270109,PF,S0007491,Y,A26634266,NaN,M0019694,D012711,MSH,ET,D012711,(131)I-MAA,0,N,256.0,NaN
2,C0000039,ENG,P,L0000039,PF,S17175117,N,A28315139,9194921.0,1926948,NaN,RXNORM,IN,1926948,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN
3,C0000039,ENG,P,L0000039,PF,S17175117,Y,A28572604,NaN,NaN,NaN,MTH,PN,NOCODE,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN
4,C0000039,ENG,P,L0000039,VC,S0007564,Y,A0016515,NaN,M0023172,D015060,MSH,MH,D015060,"1,2-Dipalmitoylphosphatidylcholine",0,N,NaN,NaN


In [105]:
mrconso.columns = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS', 'CVF', '']

In [106]:
mrconso.drop(labels=['SUPPRESS', 'CODE', 'CVF','SAUI','SDUI', ''], axis=1, inplace=True)

In [107]:
mrconso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12139019 entries, 0 to 12139018
Data columns (total 13 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   CUI     object
 1   LAT     object
 2   TS      object
 3   LUI     object
 4   STT     object
 5   SUI     object
 6   ISPREF  object
 7   AUI     object
 8   SCUI    object
 9   SAB     object
 10  TTY     object
 11  STR     object
 12  SRL     int64 
dtypes: int64(1), object(12)
memory usage: 1.2+ GB


In [108]:
mrconso[:5]

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SCUI,SAB,TTY,STR,SRL
0,C0000005,ENG,P,L0000005,PF,S0007492,Y,A26634265,M0019694,MSH,PEP,(131)I-Macroaggregated Albumin,0
1,C0000005,ENG,S,L0270109,PF,S0007491,Y,A26634266,M0019694,MSH,ET,(131)I-MAA,0
2,C0000039,ENG,P,L0000039,PF,S17175117,N,A28315139,1926948,RXNORM,IN,"1,2-dipalmitoylphosphatidylcholine",0
3,C0000039,ENG,P,L0000039,PF,S17175117,Y,A28572604,NaN,MTH,PN,"1,2-dipalmitoylphosphatidylcholine",0
4,C0000039,ENG,P,L0000039,VC,S0007564,Y,A0016515,M0023172,MSH,MH,"1,2-Dipalmitoylphosphatidylcholine",0


#### Merge - MRCONSO on Association


In [109]:
#This file contains columns that can be mapped with UMLS data
association = pd.read_csv(r'C:\Users\pc\Desktop\C-BIA\Healthcare\MeSH\XML - CSV\MeSH_association_desc.csv')

In [110]:
association['from_namespace_rectified'] = association['from_namespace']
association['to_namespace_rectified'] = association['to_namespace']

In [111]:
association['from_namespace_rectified'] = association['from_namespace_rectified'].replace('RxNorm','RXNORM')
association['from_namespace_rectified'] = association['from_namespace_rectified'].replace('MeSH','MSH')
association['from_namespace_rectified'] = association['from_namespace_rectified'].replace('SNOMED CT','SNOMEDCT_US')

In [112]:
association['to_namespace_rectified'] = association['to_namespace_rectified'].replace('RxNorm','RXNORM')
association['to_namespace_rectified'] = association['to_namespace_rectified'].replace('MeSH','MSH')
association['to_namespace_rectified'] = association['to_namespace_rectified'].replace('SNOMED CT','SNOMEDCT_US')

In [113]:
mrconsosub = mrconso[['SCUI','SAB','CUI']]

In [114]:
mrconsosub.drop_duplicates(subset=None, keep="first", inplace=True)

<ipython-input-114-fd34af682f5c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrconsosub.drop_duplicates(subset=None, keep="first", inplace=True)


In [115]:
mrconsosub.to_csv('mrconsosub.csv', index=False, header=1, encoding='utf-8')

In [116]:
asso_merge = pd.merge(association, mrconsosub, left_on=['from_code','from_namespace_rectified'] , right_on=['SCUI','SAB'],  how='left')

In [117]:
asso_merge['from_CUI'] = asso_merge['CUI']

In [118]:
asso_merge['from_SAB'] = asso_merge['SAB']

In [119]:
asso_merge['from_SCUI'] = asso_merge['SCUI']

In [120]:
asso_merge.drop(labels=['SCUI','SAB','CUI'], axis=1, inplace=True)

In [121]:
asso_merge.drop_duplicates()

,namespace,name,from_namespace,from_code,to_namespace,to_code,version,from_namespace_rectified,to_namespace_rectified,from_CUI,from_SAB,from_SCUI
0,MeSH,NRW,MeSH,M0000001,MeSH,M0353609,2021_2021.01.01,MSH,MSH,C0000699,MSH,M0000001
1,MeSH,NRW,MeSH,M0000002,MeSH,M0352201,2021_2021.01.01,MSH,MSH,C0000714,MSH,M0000002
2,MeSH,NRW,MeSH,M0000002,MeSH,M0352200,2021_2021.01.01,MSH,MSH,C0000714,MSH,M0000002
3,MeSH,NRW,MeSH,M0000004,MeSH,M0511063,2021_2021.01.01,MSH,MSH,C1959598,MSH,M0000004
4,MeSH,NRW,MeSH,M0000009,MeSH,M000615276,2021_2021.01.01,MSH,MSH,C0000739,MSH,M0000009
...,...,...,...,...,...,...,...,...,...,...,...,...
29009,MeSH,BRD,MeSH,M0593750,MeSH,M0593752,2021_2021.01.01,MSH,MSH,C0682695,MSH,M0593750
29010,MeSH,REL,MeSH,M0593750,MeSH,M0593751,2021_2021.01.01,MSH,MSH,C0682695,MSH,M0593750
29011,MeSH,BRD,MeSH,M0593772,MeSH,M0011184,2021_2021.01.01,MSH,MSH,C3850137,MSH,M0593772
29012,MeSH,NRW,MeSH,M0593772,MeSH,M000679324,2021_2021.01.01,MSH,MSH,C3850137,MSH,M0593772


In [122]:
asso_merge_final = pd.merge(asso_merge, mrconsosub, left_on=['to_code','to_namespace_rectified'] , right_on=['SCUI','SAB'],  how='left')

In [123]:
asso_merge_final['to_CUI'] = asso_merge_final['CUI'] 

In [124]:
asso_merge_final['to_SAB'] = asso_merge_final['SAB']

In [125]:
asso_merge_final['to_SCUI'] = asso_merge_final['SCUI']

In [126]:
asso_merge_final.drop(labels=['SCUI','SAB','CUI','from_SAB','to_SAB','from_SCUI','to_SCUI'], axis=1, inplace=True)

In [127]:
#vers = version['version'].astype(str)
#asso_merge_final['version'] = vers[0]
#asso_merge_final['version'] = asso_merge_final['version'].replace(asso_merge_final['version'],vers[0])

In [128]:
asso_merge_final.to_csv('MeSH_Association_Merged.csv', index=False, header=1, encoding='utf-8')

##### Move the "MeSH_Association_Merged" into the Neo4J import folder

### Creating relationships of MeSH data into UMLS model

In [12]:
import pandas as pd

In [13]:
import getpass
password = getpass.getpass("\nPlease enter the Neo4j database password to continue \n")


Please enter the Neo4j database password to continue 
········


In [14]:
from neo4j import GraphDatabase
driver=GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j',password))
session=driver.session()

In [132]:
# Create a node for each concept in the UMLS
#query = '''USING PERIODIC COMMIT 100000 LOAD CSV FROM "file:///CUIs_preferred_terms.csv" AS COLUMN CREATE (:Concept_UMLS {preferred_term:COLUMN[1], cui:COLUMN[0], semantic_type:COLUMN[2], UMLS_edition:'2021AA'})'''

#session.run(query)

In [133]:
# Create a uniqueness constraint for the CUI property of each Concept_UMLS node
#query = 'CREATE CONSTRAINT UniqueCUIforConceptConstraint ON (c:Concept_UMLS) ASSERT c.cui IS UNIQUE'
#session.run(query)

In [134]:
# Create the UMLS string nodes
#query = '''USING PERIODIC COMMIT 100000 LOAD CSV FROM "file:///str_to_CUI.csv" AS COLUMN CREATE (s:String_UMLS {string:COLUMN[0], cui:COLUMN[1], semantic_type:COLUMN[2]})'''

#session.run(query)

###### Sometimes python causes issue and the command below is not executed. In case such a thing happens , it's recommended to run the command again.

To check if these specific relations were added, search in the Neo4J browser for relationship by the name of 'NRW' , 'BRD' , 'REL'. If these 3 exist , most probably all the relationships from the MeSH_Association_Merged.csv have been added.

Use the following code to check if the relationship exist: (you can replace REL with the name of the relationship)

MATCH p=()-[r:REL]->() RETURN p LIMIT 25

In [16]:
command = '''USING PERIODIC COMMIT 100000 LOAD CSV WITH HEADERS FROM "file:///MeSH_Association_Merged.csv" AS COLUMN MATCH (c1:Concept_UMLS {cui:COLUMN.from_CUI}) MATCH (c2:Concept_UMLS {cui:COLUMN.to_CUI}) OPTIONAL MATCH (c1)-[rel]->(c2) WITH c1, c2, COLUMN, COLLECT(TYPE(rel)) AS relTypes WHERE NOT COLUMN.name IN relTypes CALL apoc.create.relationship(c1, COLUMN.name, {source:COLUMN.namespace, version:COLUMN.version}, c2) YIELD rel as newrel RETURN c1, c2, newrel;'''
session.run(command)

###### Once you reach this point , you can delete all the existing variables. It is preferable to do so if your system cannot handle executing all of these codes in one go

# Importing RxNorm into UMLS Model

To download the MeSH data you must have an account on the NLM official site 

RxNorm technical documentation - https://www.nlm.nih.gov/research/umls/rxnorm/docs/techdoc.html#s1_0

We will perform merging of two files - RXNREL.RRF and RXNCONSO.RRF.

This is necessary to get the necessary columns for further merging with MRCONSO (UMLS) so we can import our RxNorm data into our UMLS model

In [136]:
# Load RXNREL.RRF into a dataframe
rxnrel = pd.read_csv('C:/Users/pc/Desktop/C-BIA/Healthcare/RxNorm/Data/rrf/RXNREL.RRF', sep='|', header=None, encoding='utf-8')
rxnrel[:5]

C:\Users\pc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,NaN,5.0,AUI,SY,NaN,6.0,AUI,permuted_term_of,155592245.0,NaN,MSH,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,5.0,SDUI,SIB,NaN,104746.0,SDUI,NaN,154524204.0,NaN,MSH,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,5.0,SDUI,RN,NaN,609702.0,SDUI,mapped_to,154691227.0,NaN,MSH,NaN,1.0,NaN,NaN,NaN,NaN
3,NaN,5.0,AUI,SY,NaN,2666961.0,AUI,sort_version_of,155371534.0,NaN,MSH,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,5.0,AUI,SY,NaN,2681015.0,AUI,entry_version_of,155054914.0,NaN,MSH,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
rxnrel.columns = ['RXCUI1', 'RXAUI1', 'STYPE1', 'REL', 'RXCUI2', 'RXAUI2', 'STYPE2', 'RELA', 'RUI', 'SRUI', 'SAB', 'SL', 'RG☨', 'DIR☦', 'SUPPRESS', 'CVF', '']

In [138]:
rxnrel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7163685 entries, 0 to 7163684
Data columns (total 17 columns):
 #   Column    Dtype  
---  ------    -----  
 0   RXCUI1    float64
 1   RXAUI1    float64
 2   STYPE1    object 
 3   REL       object 
 4   RXCUI2    float64
 5   RXAUI2    float64
 6   STYPE2    object 
 7   RELA      object 
 8   RUI       float64
 9   SRUI      float64
 10  SAB       object 
 11  SL        float64
 12  RG☨       float64
 13  DIR☦      float64
 14  SUPPRESS  float64
 15  CVF       float64
 16            float64
dtypes: float64(12), object(5)
memory usage: 929.1+ MB


In [139]:
rxnrel[:5]

,RXCUI1,RXAUI1,STYPE1,REL,RXCUI2,RXAUI2,STYPE2,RELA,RUI,SRUI,SAB,SL,RG☨,DIR☦,SUPPRESS,CVF,
0,NaN,5.0,AUI,SY,NaN,6.0,AUI,permuted_term_of,155592245.0,NaN,MSH,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,5.0,SDUI,SIB,NaN,104746.0,SDUI,NaN,154524204.0,NaN,MSH,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,5.0,SDUI,RN,NaN,609702.0,SDUI,mapped_to,154691227.0,NaN,MSH,NaN,1.0,NaN,NaN,NaN,NaN
3,NaN,5.0,AUI,SY,NaN,2666961.0,AUI,sort_version_of,155371534.0,NaN,MSH,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,5.0,AUI,SY,NaN,2681015.0,AUI,entry_version_of,155054914.0,NaN,MSH,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
# Load RXNCONSO.RRF into a dataframe
rxnconso = pd.read_csv('C:/Users/pc/Desktop/C-BIA/Healthcare/RxNorm/Data/rrf/RXNCONSO.RRF', sep='|', header=None, encoding='utf-8')
rxnconso[:5]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,3,ENG,NaN,NaN,NaN,NaN,NaN,8717795,NaN,58488005,NaN,SNOMEDCT_US,PT,58488005,"1,4-alpha-Glucan branching enzyme",NaN,N,NaN,NaN
1,3,ENG,NaN,NaN,NaN,NaN,NaN,8717796,NaN,58488005,NaN,SNOMEDCT_US,FN,58488005,"1,4-alpha-Glucan branching enzyme (substance)",NaN,N,NaN,NaN
2,3,ENG,NaN,NaN,NaN,NaN,NaN,8717808,NaN,58488005,NaN,SNOMEDCT_US,SY,58488005,"Amylo-(1,4,6)-transglycosylase",NaN,N,NaN,NaN
3,3,ENG,NaN,NaN,NaN,NaN,NaN,8718164,NaN,58488005,NaN,SNOMEDCT_US,SY,58488005,Branching enzyme,NaN,N,NaN,NaN
4,19,ENG,NaN,NaN,NaN,NaN,NaN,10794494,NaN,112116001,NaN,SNOMEDCT_US,SY,112116001,17-hydrocorticosteroid,NaN,N,NaN,NaN


In [141]:
rxnconso.columns = ['RXCUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'RXAUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS', 'CVF', '']

In [142]:
rxnconso[:5]

,RXCUI,LAT,TS,LUI,STT,SUI,ISPREF,RXAUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,
0,3,ENG,NaN,NaN,NaN,NaN,NaN,8717795,NaN,58488005,NaN,SNOMEDCT_US,PT,58488005,"1,4-alpha-Glucan branching enzyme",NaN,N,NaN,NaN
1,3,ENG,NaN,NaN,NaN,NaN,NaN,8717796,NaN,58488005,NaN,SNOMEDCT_US,FN,58488005,"1,4-alpha-Glucan branching enzyme (substance)",NaN,N,NaN,NaN
2,3,ENG,NaN,NaN,NaN,NaN,NaN,8717808,NaN,58488005,NaN,SNOMEDCT_US,SY,58488005,"Amylo-(1,4,6)-transglycosylase",NaN,N,NaN,NaN
3,3,ENG,NaN,NaN,NaN,NaN,NaN,8718164,NaN,58488005,NaN,SNOMEDCT_US,SY,58488005,Branching enzyme,NaN,N,NaN,NaN
4,19,ENG,NaN,NaN,NaN,NaN,NaN,10794494,NaN,112116001,NaN,SNOMEDCT_US,SY,112116001,17-hydrocorticosteroid,NaN,N,NaN,NaN


In [143]:
rxnconso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093950 entries, 0 to 1093949
Data columns (total 19 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   RXCUI     1093950 non-null  int64  
 1   LAT       1093950 non-null  object 
 2   TS        0 non-null        float64
 3   LUI       0 non-null        float64
 4   STT       0 non-null        float64
 5   SUI       0 non-null        float64
 6   ISPREF    0 non-null        float64
 7   RXAUI     1093950 non-null  int64  
 8   SAUI      318126 non-null   float64
 9   SCUI      469292 non-null   object 
 10  SDUI      24794 non-null    object 
 11  SAB       1093950 non-null  object 
 12  TTY       1093950 non-null  object 
 13  CODE      1093949 non-null  object 
 14  STR       1093950 non-null  object 
 15  SRL       0 non-null        float64
 16  SUPPRESS  1093950 non-null  object 
 17  CVF       233718 non-null   float64
 18            0 non-null        float64
dtypes: float64(9), int64(

In [144]:
rxnrel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7163685 entries, 0 to 7163684
Data columns (total 17 columns):
 #   Column    Dtype  
---  ------    -----  
 0   RXCUI1    float64
 1   RXAUI1    float64
 2   STYPE1    object 
 3   REL       object 
 4   RXCUI2    float64
 5   RXAUI2    float64
 6   STYPE2    object 
 7   RELA      object 
 8   RUI       float64
 9   SRUI      float64
 10  SAB       object 
 11  SL        float64
 12  RG☨       float64
 13  DIR☦      float64
 14  SUPPRESS  float64
 15  CVF       float64
 16            float64
dtypes: float64(12), object(5)
memory usage: 929.1+ MB


In [145]:
check = ['788773']
rxnconso[rxnconso.RXAUI.isin(check)]

C:\Users\pc\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,RXCUI,LAT,TS,LUI,STT,SUI,ISPREF,RXAUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,


In [146]:
rxnrelsub = rxnrel
rxnconsosub = rxnconso

In [147]:
rxnconsosub.drop(labels=['TS', 'STT', 'ISPREF', 'SRL', 'LUI', 'SUI', 'SUPPRESS', 'CVF','SAUI','SDUI', ''], axis=1, inplace=True)

In [148]:
rxnrelsub.drop(labels=['DIR☦', 'RG☨', 'SRUI', 'SL', 'SUPPRESS', 'CVF', ''], axis=1, inplace=True)

In [149]:
rxnrelsub[:5]

,RXCUI1,RXAUI1,STYPE1,REL,RXCUI2,RXAUI2,STYPE2,RELA,RUI,SAB
0,NaN,5.0,AUI,SY,NaN,6.0,AUI,permuted_term_of,155592245.0,MSH
1,NaN,5.0,SDUI,SIB,NaN,104746.0,SDUI,NaN,154524204.0,MSH
2,NaN,5.0,SDUI,RN,NaN,609702.0,SDUI,mapped_to,154691227.0,MSH
3,NaN,5.0,AUI,SY,NaN,2666961.0,AUI,sort_version_of,155371534.0,MSH
4,NaN,5.0,AUI,SY,NaN,2681015.0,AUI,entry_version_of,155054914.0,MSH


In [150]:
rxnconsosub[:5]

,RXCUI,LAT,RXAUI,SCUI,SAB,TTY,CODE,STR
0,3,ENG,8717795,58488005,SNOMEDCT_US,PT,58488005,"1,4-alpha-Glucan branching enzyme"
1,3,ENG,8717796,58488005,SNOMEDCT_US,FN,58488005,"1,4-alpha-Glucan branching enzyme (substance)"
2,3,ENG,8717808,58488005,SNOMEDCT_US,SY,58488005,"Amylo-(1,4,6)-transglycosylase"
3,3,ENG,8718164,58488005,SNOMEDCT_US,SY,58488005,Branching enzyme
4,19,ENG,10794494,112116001,SNOMEDCT_US,SY,112116001,17-hydrocorticosteroid


In [151]:
check = ['2666961']
rxnconsosub[rxnconsosub.RXAUI.isin(check)]

C:\Users\pc\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,RXCUI,LAT,RXAUI,SCUI,SAB,TTY,CODE,STR


In [152]:
rxnrel[rxnrel.RXCUI1.notnull()]

,RXCUI1,RXAUI1,STYPE1,REL,RXCUI2,RXAUI2,STYPE2,RELA,RUI,SAB
5714011,38.0,NaN,CUI,RB,1760.0,NaN,CUI,has_tradename,4696871.0,RXNORM
5714012,38.0,NaN,CUI,RO,105050.0,NaN,CUI,has_ingredient,4343918.0,RXNORM
5714013,38.0,NaN,CUI,RO,105445.0,NaN,CUI,has_ingredient,4229336.0,RXNORM
5714014,38.0,NaN,CUI,RO,105446.0,NaN,CUI,has_ingredient,3798489.0,RXNORM
5714015,38.0,NaN,CUI,RO,105447.0,NaN,CUI,has_ingredient,4423580.0,RXNORM
...,...,...,...,...,...,...,...,...,...,...
7163680,2567330.0,NaN,CUI,RB,669081.0,NaN,CUI,has_tradename,161706809.0,RXNORM
7163681,2567330.0,NaN,CUI,RO,2567326.0,NaN,CUI,ingredient_of,161706801.0,RXNORM
7163682,2567330.0,NaN,CUI,RO,2567327.0,NaN,CUI,constitutes,161706805.0,RXNORM
7163683,2567330.0,NaN,CUI,RB,2567328.0,NaN,CUI,inverse_isa,161706813.0,RXNORM


In [153]:
check = ['38.0']
rxnconsosub[rxnconsosub.RXCUI.isin(check)]

C:\Users\pc\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,RXCUI,LAT,RXAUI,SCUI,SAB,TTY,CODE,STR


In [154]:
#rxnconsosub = rxnconsosub.astype({'RXCUI': 'float', 'RXAUI': 'float'})

In [155]:
rxnconsosub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093950 entries, 0 to 1093949
Data columns (total 8 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   RXCUI   1093950 non-null  int64 
 1   LAT     1093950 non-null  object
 2   RXAUI   1093950 non-null  int64 
 3   SCUI    469292 non-null   object
 4   SAB     1093950 non-null  object
 5   TTY     1093950 non-null  object
 6   CODE    1093949 non-null  object
 7   STR     1093950 non-null  object
dtypes: int64(2), object(6)
memory usage: 66.8+ MB


In [156]:
check = ['788773']
rxnrelsub[rxnrelsub.RXAUI1.isin(check)]

C:\Users\pc\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,RXCUI1,RXAUI1,STYPE1,REL,RXCUI2,RXAUI2,STYPE2,RELA,RUI,SAB


#### Merging based on SCUI column for RXNREL and RXNCONSO 

In [157]:
rxnrelcui = rxnrelsub[rxnrelsub.RXCUI1.notnull()]

In [158]:
rxnrelaui = rxnrelsub[rxnrelsub.RXAUI1.notnull()]

In [159]:
asso_merge1 = pd.merge(rxnrelaui, rxnconsosub, left_on=['RXAUI1','SAB'] , right_on=['RXAUI','SAB'],  how='left')

In [160]:
asso_merge1.drop_duplicates(subset=None, keep="first", inplace=True)

In [161]:
asso_merge1['from_SCUI'] = asso_merge1['SCUI']
asso_merge1['from_namespace'] = asso_merge1['SAB']

In [162]:
asso_merge1.drop(labels=['RXCUI', 'LAT', 'RXAUI', 'SCUI', 'TTY', 'STR', 'CODE'], axis=1, inplace=True)

In [163]:
asso_merge2 = pd.merge(asso_merge1, rxnconsosub, left_on=['RXAUI2','SAB'] , right_on=['RXAUI','SAB'],  how='left')

In [164]:
asso_merge2.drop_duplicates(subset=None, keep="first", inplace=True)

In [165]:
asso_merge2['to_SCUI'] = asso_merge2['SCUI']
asso_merge2['to_namespace'] = asso_merge2['SAB']

In [166]:
#asso_merge2.drop(labels=['RXCUI', 'LAT', 'RXAUI', 'SCUI', 'TTY', 'STR', 'CODE'], axis=1, inplace=True)

In [167]:
asso_merge3 = pd.merge(rxnrelcui, rxnconsosub, left_on=['RXCUI1','SAB'] , right_on=['RXCUI','SAB'],  how='left')

In [168]:
asso_merge3.drop_duplicates(subset=None, keep="first", inplace=True)

In [169]:
asso_merge3['from_SCUI'] = asso_merge3['SCUI']
asso_merge3['from_namespace'] = asso_merge3['SAB']

In [170]:
asso_merge3.drop(labels=['RXCUI', 'LAT', 'RXAUI', 'SCUI', 'TTY', 'STR', 'CODE'], axis=1, inplace=True)

In [171]:
asso_merge4 = pd.merge(asso_merge3, rxnconsosub, left_on=['RXCUI2','SAB'] , right_on=['RXCUI','SAB'],  how='left')

In [172]:
asso_merge4.drop_duplicates(subset=None, keep="first", inplace=True)

In [173]:
asso_merge4['to_SCUI'] = asso_merge4['SCUI']
asso_merge4['to_namespace'] = asso_merge4['SAB']

In [174]:
asso_mergef = pd.concat([asso_merge2,asso_merge4], ignore_index=True)

In [175]:
asso_mergef.drop(labels=['RXCUI', 'LAT', 'RXAUI', 'SCUI', 'TTY', 'STR', 'CODE'], axis=1, inplace=True)

In [176]:
asso_mergef.drop_duplicates(subset=None, keep="first", inplace=True)

In [177]:
asso_mergef = asso_mergef[asso_mergef.from_SCUI.notnull()]
asso_mergef = asso_mergef[asso_mergef.to_SCUI.notnull()]

In [178]:
asso_mergef[:5]

,RXCUI1,RXAUI1,STYPE1,REL,RXCUI2,RXAUI2,STYPE2,RELA,RUI,SAB,from_SCUI,from_namespace,to_SCUI,to_namespace
0,NaN,5.0,AUI,SY,NaN,6.0,AUI,permuted_term_of,155592245.0,MSH,M0023172,MSH,M0023172,MSH
1,NaN,5.0,SDUI,SIB,NaN,104746.0,SDUI,NaN,154524204.0,MSH,M0023172,MSH,M0006471,MSH
2,NaN,5.0,SDUI,RN,NaN,609702.0,SDUI,mapped_to,154691227.0,MSH,M0023172,MSH,M0217563,MSH
3,NaN,5.0,AUI,SY,NaN,2666961.0,AUI,sort_version_of,155371534.0,MSH,M0023172,MSH,M0023172,MSH
4,NaN,5.0,AUI,SY,NaN,2681015.0,AUI,entry_version_of,155054914.0,MSH,M0023172,MSH,M0023172,MSH


In [179]:
asso_mergef['from_namespace_rectified'] = asso_mergef['from_namespace'].replace('RxNorm','RXNORM')
asso_mergef['from_namespace_rectified'] = asso_mergef['from_namespace'].replace('MeSH','MSH')
asso_mergef['from_namespace_rectified'] = asso_mergef['from_namespace'].replace('SNOMED CT','SNOMEDCT_US')

In [180]:
asso_mergef['to_namespace_rectified'] = asso_mergef['to_namespace'].replace('RxNorm','RXNORM')
asso_mergef['to_namespace_rectified'] = asso_mergef['to_namespace'].replace('MeSH','MSH')
asso_mergef['to_namespace_rectified'] = asso_mergef['to_namespace'].replace('SNOMED CT','SNOMEDCT_US')

In [181]:
asso_mergef.to_csv('Association_Merged_RXN_incomplete.csv', index=False, header=1, encoding='utf-8')

### Merging with MRCONSO 

In [182]:
# Load MRCONSO.RRF into a dataframe
mrconso = pd.read_csv('C:/Users/pc/Desktop/C-BIA/Healthcare/UMLS/UMLS_Data/2021AA/META/MRCONSO.RRF', sep='|', header=None, encoding='utf-8')
mrconso[:5]

C:\Users\pc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,C0000005,ENG,P,L0000005,PF,S0007492,Y,A26634265,NaN,M0019694,D012711,MSH,PEP,D012711,(131)I-Macroaggregated Albumin,0,N,256.0,NaN
1,C0000005,ENG,S,L0270109,PF,S0007491,Y,A26634266,NaN,M0019694,D012711,MSH,ET,D012711,(131)I-MAA,0,N,256.0,NaN
2,C0000039,ENG,P,L0000039,PF,S17175117,N,A28315139,9194921.0,1926948,NaN,RXNORM,IN,1926948,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN
3,C0000039,ENG,P,L0000039,PF,S17175117,Y,A28572604,NaN,NaN,NaN,MTH,PN,NOCODE,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN
4,C0000039,ENG,P,L0000039,VC,S0007564,Y,A0016515,NaN,M0023172,D015060,MSH,MH,D015060,"1,2-Dipalmitoylphosphatidylcholine",0,N,NaN,NaN


In [183]:
mrconso.columns = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS', 'CVF', '']

In [184]:
mrconsosub = mrconso[['SCUI','CUI','SAB']]
mrconsosub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12139019 entries, 0 to 12139018
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   SCUI    object
 1   CUI     object
 2   SAB     object
dtypes: object(3)
memory usage: 277.8+ MB


In [185]:
asso_mergef_sub = asso_mergef

In [186]:
asso_mergef2 = pd.merge(asso_mergef_sub, mrconsosub, left_on=['from_SCUI','from_namespace_rectified'] , right_on=['SCUI','SAB'],  how='left')

In [187]:
asso_mergef2.drop_duplicates(subset=None, keep="first", inplace=True)

In [188]:
asso_mergef2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1670087 entries, 0 to 3136153
Data columns (total 19 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   RXCUI1                    1449674 non-null  float64
 1   RXAUI1                    220413 non-null   float64
 2   STYPE1                    1670087 non-null  object 
 3   REL                       1670087 non-null  object 
 4   RXCUI2                    1449674 non-null  float64
 5   RXAUI2                    220413 non-null   float64
 6   STYPE2                    1670087 non-null  object 
 7   RELA                      1527748 non-null  object 
 8   RUI                       1552893 non-null  float64
 9   SAB_x                     1670087 non-null  object 
 10  from_SCUI                 1670087 non-null  object 
 11  from_namespace            1670087 non-null  object 
 12  to_SCUI                   1670087 non-null  object 
 13  to_namespace              1

In [189]:
asso_mergef2['from_CUI'] = asso_mergef2['CUI']

In [190]:
asso_mergef2.drop(labels=['CUI','SCUI'], axis=1, inplace=True)

In [191]:
asso_mergef2 = pd.merge(asso_mergef2, mrconsosub, left_on=['to_SCUI','to_namespace_rectified'] , right_on=['SCUI','SAB'],  how='left')

In [192]:
asso_mergef2.drop_duplicates(subset=None, keep="first", inplace=True)

In [193]:
asso_mergef2['to_CUI'] = asso_mergef2['CUI']

In [194]:
asso_mergef2.drop(labels=['CUI','SCUI'], axis=1, inplace=True)

In [195]:
asso_mergef2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1672503 entries, 0 to 3146751
Data columns (total 20 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   RXCUI1                    1449674 non-null  float64
 1   RXAUI1                    222829 non-null   float64
 2   STYPE1                    1672503 non-null  object 
 3   REL                       1672503 non-null  object 
 4   RXCUI2                    1449674 non-null  float64
 5   RXAUI2                    222829 non-null   float64
 6   STYPE2                    1672503 non-null  object 
 7   RELA                      1529945 non-null  object 
 8   RUI                       1555309 non-null  float64
 9   SAB_x                     1672503 non-null  object 
 10  from_SCUI                 1672503 non-null  object 
 11  from_namespace            1672503 non-null  object 
 12  to_SCUI                   1672503 non-null  object 
 13  to_namespace              1

In [196]:
asso_mergef2.drop_duplicates(subset=None, keep="first", inplace=True)

In [197]:
asso_mergef2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1672503 entries, 0 to 3146751
Data columns (total 20 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   RXCUI1                    1449674 non-null  float64
 1   RXAUI1                    222829 non-null   float64
 2   STYPE1                    1672503 non-null  object 
 3   REL                       1672503 non-null  object 
 4   RXCUI2                    1449674 non-null  float64
 5   RXAUI2                    222829 non-null   float64
 6   STYPE2                    1672503 non-null  object 
 7   RELA                      1529945 non-null  object 
 8   RUI                       1555309 non-null  float64
 9   SAB_x                     1672503 non-null  object 
 10  from_SCUI                 1672503 non-null  object 
 11  from_namespace            1672503 non-null  object 
 12  to_SCUI                   1672503 non-null  object 
 13  to_namespace              1

In [199]:
asso_mergef2['version'] = '2021.09.07.21AA'
asso_mergef2['namespace'] = 'RxNorm'
asso_mergef2['name'] = asso_mergef2['REL']
asso_mergef2[:5]

,RXCUI1,RXAUI1,STYPE1,REL,RXCUI2,RXAUI2,STYPE2,RELA,RUI,SAB_x,...,to_namespace,from_namespace_rectified,to_namespace_rectified,SAB_y,from_CUI,SAB,to_CUI,version,namespace,name
0,NaN,5.0,AUI,SY,NaN,6.0,AUI,permuted_term_of,155592245.0,MSH,...,MSH,MSH,MSH,MSH,C0000039,MSH,C0000039,2021.09.07.21AA,RxNorm,SY
14,NaN,5.0,SDUI,SIB,NaN,104746.0,SDUI,NaN,154524204.0,MSH,...,MSH,MSH,MSH,MSH,C0000039,MSH,C0012456,2021.09.07.21AA,RxNorm,SIB
25,NaN,5.0,SDUI,RN,NaN,609702.0,SDUI,mapped_to,154691227.0,MSH,...,MSH,MSH,MSH,MSH,C0000039,MSH,C0216971,2021.09.07.21AA,RxNorm,RN
27,NaN,5.0,AUI,SY,NaN,2666961.0,AUI,sort_version_of,155371534.0,MSH,...,MSH,MSH,MSH,MSH,C0000039,MSH,C0000039,2021.09.07.21AA,RxNorm,SY
41,NaN,5.0,AUI,SY,NaN,2681015.0,AUI,entry_version_of,155054914.0,MSH,...,MSH,MSH,MSH,MSH,C0000039,MSH,C0000039,2021.09.07.21AA,RxNorm,SY


In [200]:
asso_mergef2.to_csv('Association_Merged_RXN_check.csv', index=False, header=1, encoding='utf-8')

##### Move the "Association_Merged_RXN_check" into the Neo4J import folder

## Importing in Neo4J 

In [1]:
import pandas as pd

In [2]:
import getpass
password = getpass.getpass("\nPlease enter the Neo4j database password to continue \n")


Please enter the Neo4j database password to continue 
········


In [3]:
from neo4j import GraphDatabase
driver=GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j',password))
session=driver.session()

In [204]:
# Create a node for each concept in the UMLS
#query = '''USING PERIODIC COMMIT 100000 LOAD CSV FROM "file:///CUIs_preferred_terms.csv" AS COLUMN CREATE (:Concept_UMLS {preferred_term:COLUMN[1], cui:COLUMN[0], semantic_type:COLUMN[2], UMLS_edition:'2021AA'})'''

#session.run(query)

In [205]:
# Create a uniqueness constraint for the CUI property of each Concept_UMLS node
#query = 'CREATE CONSTRAINT UniqueCUIforConceptConstraint ON (c:Concept_UMLS) ASSERT c.cui IS UNIQUE'
#session.run(query)

In [206]:
# Create the UMLS string nodes
#query = '''USING PERIODIC COMMIT 100000 LOAD CSV FROM "file:///str_to_CUI.csv" AS COLUMN CREATE (s:String_UMLS {string:COLUMN[0], cui:COLUMN[1], semantic_type:COLUMN[2]})'''

#session.run(query)

###### Sometimes python causes issue and the command below is not executed. In case such a thing happens , it's recommended to run the cell/command again.

To check if these specific relations were added, search in the Neo4J browser for relationship by the name of 'CHD' , 'SIB' , 'SY'. If these 3 exist , most probably all the relationships from the Association_Merged_RXN.csv have been added.

Use the following code to check if the relationship exist: (you can replace CHD with the name of the relationship)

MATCH p=()-[r:CHD]->() RETURN p LIMIT 25

In [5]:
command = '''USING PERIODIC COMMIT 1000000 LOAD CSV WITH HEADERS FROM "file:///Association_Merged_RXN_check.csv" AS COLUMN MATCH (c1:Concept_UMLS {cui:COLUMN.from_CUI}) MATCH (c2:Concept_UMLS {cui:COLUMN.to_CUI}) OPTIONAL MATCH (c1)-[rel]->(c2) WITH c1, c2, COLUMN, COLLECT(TYPE(rel)) AS relTypes WHERE NOT COLUMN.name IN relTypes CALL apoc.create.relationship(c1, COLUMN.name, {source:COLUMN.namespace, version:COLUMN.version, REL_Additional:COLUMN.RELA}, c2) YIELD rel as newrel RETURN c1, c2, newrel;'''
session.run(command)

### Regarding the Relationships created after executing the code

The following link contains information about what the relationship names stand for: https://www.nlm.nih.gov/research/umls/knowledge_sources/metathesaurus/release/abbreviations.html#mrdoc_REL

